In [1]:
import sys
# custom utilies for displaying animation, collecting rollouts and more
sys.path.append("~/Adventures/CustomA2C")

%load_ext autoreload
%autoreload 2

In [13]:
import matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from parallelEnv import parallelEnv
from helpers import sample_trajectories
from a2c_agent import a2c_agent

agent = a2c_agent(8, 4)
envs = parallelEnv('LunarLander-v2', n=4)
device = torch.device("cpu")
print(np.arange(envs.action_space.n))
#states, actions, rewards = sample_trajectories(envs, agent, tmax=200)
#print(rewards)

started processes:  [2883, 2884, 2885, 2886]
[0 1 2 3]


## Training

In [24]:
print(agent.named_parameters)


## training params ##
ROLLOUT_LENGTH = 300
NUM_EPOCHS = 60
DISCOUNT = 0.99
BETA = 0.01 #entropy term
CRITIC_LOSS_COEFF = 1.0 #critic term
GRADIENT_CLIP = 0.5
LEARNING_RATE = 0.0003
optimizer = torch.optim.RMSprop(agent.parameters(),lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.98 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)


## LOGGING SETUP ##s
import logging
import os
#logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s: %(message)s')
logger = logging.getLogger() 
lognum = 1
#logger.setLevel(logging.INFO)
fh = logging.FileHandler('./log/%s-%s.txt' % ('train', lognum))
fh.setFormatter(logging.Formatter('%(asctime)s - %(name)s: %(message)s'))
fh.setLevel(logging.INFO)
logger.addHandler(fh)


def step():
    envs.reset()
    #take initial random steps
    for i in range(5):
        s, r, done, _ = envs.step(np.random.choice(np.arange(envs.action_space.n), envs.num_envs))
        s, r, done, _ = envs.step([0]* envs.num_envs) #wait a frame
    
    for ep in range(NUM_EPOCHS):
        storage = {"states": [], "values": [], "log_probs": [],
                    "actions":[], "entropy":[], "rewards":[], "dones":[]}
        total_rewards = 0.0
        
        for t in range(ROLLOUT_LENGTH + 1): #one more step for ending!
            # step throguh agent
            s_tensor = torch.from_numpy(s).float().to(device)
            preds = agent.forward(s_tensor)
            actions = preds["actions"].detach().numpy()

            storage["states"].append(s_tensor)
            storage["values"].append(preds["values"])
            storage["log_probs"].append(preds["log_probs"])
            storage["actions"].append(preds["actions"])
            storage["entropy"].append(preds["entropy"])

            # step through env
            s, r, done, _ = envs.step(actions)
            storage["rewards"].append(torch.tensor(r))
            storage["dones"].append(torch.tensor(1-done))
            total_rewards += r.mean()
        
        total_rewards /= ROLLOUT_LENGTH

        storage["advantages"] = [torch.zeros((envs.num_envs, 1)).to(device)]*(ROLLOUT_LENGTH + 1)
        for i in reversed(range(ROLLOUT_LENGTH)):
            td_error = storage["rewards"][i] + DISCOUNT * storage["values"][i+1] - storage["values"][i]
            td_error.detach()
            storage["advantages"][i] = td_error
        
        # collect
        log_probs = torch.stack(storage["log_probs"][:-1])
        advantages = torch.stack(storage["advantages"][:-1])
        entropy = torch.stack(storage["entropy"][:-1])
        
        # calculate losses
        actor_loss = -(log_probs * advantages).mean() #this part is ascent!
        critic_loss = 0.5 * (advantages).pow(2).mean() #this is descent
        entropy_loss = entropy.mean()
        L = actor_loss + CRITIC_LOSS_COEFF * critic_loss - BETA * entropy_loss

        # backprop
        optimizer.zero_grad()
        L.backward()
        nn.utils.clip_grad_norm_(agent.parameters(), GRADIENT_CLIP)
        optimizer.step()
        scheduler.step()
        print("loss:", L)
        logger.info("Episode: {0:d}, loss: {1:f}, rewards: {2:f}, lr: {3:f}".format(ep+1, L, total_rewards, optimizer.param_groups[0]['lr']))
        if (ep+1)%20 ==0 :
            print("Episode: {0:d}, loss: {1:f}, rewards: {2:f}, lr: {3:f}".format(ep+1, L, total_rewards, optimizer.param_groups[0]['lr']))



<bound method Module.named_parameters of a2c_agent(
  (shared_network): SharedNetwork(
    (layers): ModuleList(
      (0): Linear(in_features=8, out_features=64, bias=True)
      (1): Linear(in_features=64, out_features=64, bias=True)
    )
  )
  (critic_network): CriticNetwork(
    (layers): ModuleList(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): Linear(in_features=32, out_features=1, bias=True)
    )
  )
  (actor_network): ActorNetwork(
    (layers): ModuleList(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): Linear(in_features=32, out_features=4, bias=True)
    )
  )
)>


In [25]:
step()
agent.save('pretraied-model-1')

2023-07-10 19:27:41,010 - root - INFO: Episode: 1, loss: 16.953126, rewards: -1.045049, lr: 0.000294


loss: tensor(16.9531, dtype=torch.float64, grad_fn=<SubBackward0>)


2023-07-10 19:27:41,531 - root - INFO: Episode: 2, loss: 16.028826, rewards: -0.112916, lr: 0.000288


loss: tensor(16.0288, dtype=torch.float64, grad_fn=<SubBackward0>)


2023-07-10 19:27:41,938 - root - INFO: Episode: 3, loss: 20.024970, rewards: -0.579440, lr: 0.000282


...

loss: tensor(10.5917, dtype=torch.float64, grad_fn=<SubBackward0>)


2023-07-10 19:28:23,549 - root - INFO: Episode: 59, loss: 14.020878, rewards: -0.334791, lr: 0.000091


loss: tensor(14.0209, dtype=torch.float64, grad_fn=<SubBackward0>)


2023-07-10 19:28:24,747 - root - INFO: Episode: 60, loss: 12.185579, rewards: -0.394047, lr: 0.000089


loss: tensor(12.1856, dtype=torch.float64, grad_fn=<SubBackward0>)
Episode: 60, loss: 12.185579, rewards: -0.394047, lr: 0.000089


In [30]:
import gymnasium
env = gymnasium.make('LunarLander-v2',render_mode="human")
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)
env.reset()

agent.load('pretraied-model-1')

#take initial random steps
for i in range(5):
    s, r, done, _, info = env.step(np.random.choice(np.arange(envs.action_space.n)))
    s, r, done, _, info = env.step(0) #wait a frame

agent.eval()
with torch.no_grad():
    for t in range(350):
        # convert observations to torch
        s = np.asarray(s)
        s_tensor = torch.from_numpy(s).float().to(device)
        preds = agent.forward(s_tensor)
        actions = preds["actions"].detach().numpy()

        # step env
        s, r, done,  _, info= env.step(actions)
        if(done): break

agent.train()
env.close()


State shape:  (8,)
Number of actions:  4
